In [2]:
from ortools.linear_solver import pywraplp
import numpy as np
import pandas as pd
from random import uniform


# Funções utilizadas

def db2pow(ydB):
    """Função que converte uma entrada em decibels (dB) em potência (W)"""
    return np.power(10,(ydB/10))

def pow2db(ypow):
    """Função que converte uma entrada em potência (W) em decibels (dB)"""
    return 10*np.log10(ypow)

def Canal(distancia,frequencia = 0.9,LOS=False):
    """ Função que retorna o canal em potência de um usuário dada a distância em metros e a frequencia da portadora em GHz"""
    if LOS == False:
        h = 22*np.log10(distancia/1000) + 28 + 20*np.log10(frequencia)
    else:
        h = 36.7*np.log10(distancia/1000) + 22.7+ 26*np.log10(frequencia)
    #h = 38+30*np.log10(distancia/1000)
    return (abs(db2pow(-h*(np.random.rayleigh())))**2)

def Otimizar_NOMA(Canal1,Canal2,No,Pt, Banda, R1_min,R2_min):
    """Função para otimizar uma transmissão NOMA.

    Canal 1 = potencia de atenuação do canal referente ao usuário proximo (W)
    Canal 2 = potencia de atenuação do canal referente ao usuário distante (W)
    No      = Figura de ruído presente no sistema (W)
    Pt      = Potencia de Transmissão (W)
    Banda   = Largura de Banda de transmissão (Hz)
    R1_min  = Throughput Alvo para o usuário 1 (Bps/Hz)
    """
    # Criando o Solver
    solver = pywraplp.Solver('LinearProgrammingExample',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    solver = pywraplp.Solver.CreateSolver('GLOP')
    X = solver.NumVar(-solver.infinity(),solver.infinity(),'x')
    Y_line = solver.NumVar(-solver.infinity(),solver.infinity(), 'y_line')
    
    #Constantes
    A = Pt*(Canal1)
    B = Pt*(Canal2)

 


    #Constraint 1
    solver.Add(X >= (2**(R1_min/Banda) - 1))

    # Constraint 2: X - AB Y_line + ANo Y_line + (A/B) = 0
    solver.Add(X - (A*Y_line) - (A*No*Y_line/B) +(A/B) == 0)

    # Constraint 3: R2 >0
    solver.Add(Y_line <=(1/No))
    #solver.Add(Y_line <= (1/((No*2**(-R2_min/Banda)))))
    # Constraint 4: alpha <= 0.5
    solver.Add(X*No/A <= 0.5)
    # Constraint 5: alpha >= 0
    solver.Add(X*No/A >= 0)
    solver.Minimize(Y_line)
    status = solver.Solve()

        
        
    alpha = X.solution_value()*No/A
    R2 = Banda*np.log2(1+(B*(1-alpha)/((B*alpha)+No)))
    R1 = Banda*np.log2(1+(A*alpha/No))
    SNR2= (B*(1-alpha)/((B*alpha)+No))
    #print(pow2db(SNR2))
    SNR1= (A*alpha/No)
    #print(pow2db(SNR1))
    if round(R1) >= R1_min:
        if R2 >= R2_min:
            ans = (R1+R2)*1
                
        else:
            ans = 0
            R2 = 0
            R1 = R1_min
    else:
        ans = 0
        R2 = 0
        R1 = R1_min
    
            
    return [ans,R1,R2,alpha, SNR1,SNR2]



In [24]:
Near_User = Canal(15,0.9)
Far_User = Canal(25,0.9)
No     = db2pow(9)
Pt     = (10**(-3))*db2pow(44)
Banda  = 1   # 1MHz
R1_min = 1
Saida = Otimizar_NOMA(Near_User,Far_User,No,Pt,Banda,5,3)
print(Saida)
A = Pt*(Near_User)
R_OMA_NU  = np.log2(1+(A/No))
print(Saida[1])


[9.487672590173561, 5.0, 4.4876725901735615, 0.04455779493726332, 31.0, 21.43489598545646]
5.0


In [4]:
Near_User

6.385243788449676

In [5]:

Far_User

29994.680031013264

In [6]:
Services = {'Netflix_4k':25,'Youtube_4k': 15,'Netflix_Hd':5,'Youtube_1080': 4,'Online_Gaming':3,'Youtube_720': 2.5, 'Music_Streaming':0.5}

Services

{'Netflix_4k': 25,
 'Youtube_4k': 15,
 'Netflix_Hd': 5,
 'Youtube_1080': 4,
 'Online_Gaming': 3,
 'Youtube_720': 2.5,
 'Music_Streaming': 0.5}

In [15]:
Near_User = Canal(32,0.9)
Far_User = Canal(200,0.9)
No     = db2pow(9)
Banda  = 1  # 1MHz
Services = {'Netflix_4k':25/Banda,'Youtube_4k': 15/Banda,'Netflix_Hd':5/Banda,'Youtube_1080': 4/Banda,'Online_Gaming':3/Banda,'Youtube_720': 2.5/Banda, 'Music_Streaming':0.5/Banda}
DF = pd.DataFrame(columns=['Distancia 1','SNR 1','Distancia 2','SNR 2','R1_min','R1_OMA','R2','R2_OMA','Alpha','Both QoS Satisfied','QoS 1', 'QoS 2'])
Pt     = (10**(-3))*db2pow(44)
x =Services.items()
for Qos1 in range(1,20):
    #print(f'{item}: {Services[item]}')
    R1_min = Qos1
    #print(f'R1_min = {R1_min}')
    for QoS2 in range(1,2):    
        R2_min = QoS2
        #print(f'R2_min = {R2_min}')
        for dist1 in range(1,35):
            Near_User = Canal(dist1,2,False)
            A = Pt*(Near_User)
            R_OMA_NU  = np.log2(1+(A/No))
            for dist2 in range(dist1,75):
                Far_User = Canal(dist2,2,False)
                B = Pt*(Far_User)
                R_OMA_FU  = np.log2(1+(B/No))
                Saida = Otimizar_NOMA(Near_User,Far_User,No,Pt,Banda,R1_min,1)
                if  Saida[1]> R2_min and Saida[0]>=R1_min:
                    DF = DF.append({'Distancia 1': dist1,'SNR 1': pow2db(Saida[4]), 'Distancia 2': dist2,'SNR 2': pow2db(Saida[5]), 'R1_min': Saida[1],'R1_OMA': R_OMA_NU, 'R2': Saida[2]/Banda,'R2_OMA': R_OMA_FU, 'Alpha': Saida[3],'Both QoS Satisfied': 1,'QoS 1':R1_min/Banda, 'QoS 2':R2_min/Banda },ignore_index=True)
                else:
                    DF = DF.append({'Distancia 1': dist1,'SNR 1': pow2db(Saida[4]), 'Distancia 2': dist2, 'SNR 2': pow2db(Saida[5]),'R1_min': Saida[1],'R1_OMA': R_OMA_NU, 'R2': Saida[2]/Banda,'R2_OMA': R_OMA_FU, 'Alpha': Saida[3],'Both QoS Satisfied': 0,'QoS 1':R1_min/Banda, 'QoS 2':R2_min/Banda },ignore_index=True)


DF

C:\Users\guga_\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log10
  from ipykernel import kernelapp as app


,Distancia 1,SNR 1,Distancia 2,SNR 2,R1_min,R1_OMA,R2,R2_OMA,Alpha,Both QoS Satisfied,QoS 1,QoS 2
0,1.0,9.643275e-16,1.0,134.756050,1.0,49.728647,44.764991,44.811982,1.071977e-15,0.0,1.0,1.0
1,1.0,9.643275e-16,2.0,112.533905,1.0,49.728647,37.382954,37.383231,1.071977e-15,0.0,1.0,1.0
2,1.0,9.643275e-16,3.0,128.918758,1.0,49.728647,42.825884,42.837992,1.071977e-15,0.0,1.0,1.0
3,1.0,9.643275e-16,4.0,86.480921,1.0,49.728647,28.728340,28.728341,1.071977e-15,0.0,1.0,1.0
4,1.0,9.643275e-16,5.0,36.759485,1.0,49.728647,12.211541,12.211541,1.071977e-15,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
37140,34.0,-inf,70.0,-4.101537,19.0,1.475181,0.000000,0.473950,0.000000e+00,0.0,19.0,1.0
37141,34.0,-inf,71.0,-25.799499,19.0,1.475181,0.000000,0.003790,0.000000e+00,0.0,19.0,1.0
37142,34.0,-inf,72.0,-13.205982,19.0,1.475181,0.000000,0.067359,0.000000e+00,0.0,19.0,1.0
37143,34.0,-inf,73.0,-12.535217,19.0,1.475181,0.000000,0.078309,0.000000e+00,0.0,19.0,1.0


In [26]:
DF.to_csv('NOMA13.csv')

In [9]:
Sumrate = DF['R1_min']+DF['R2']
DF['Sumrate'] = Sumrate
#DF['Diff R2'] = DF['R2_max'] - DF['R2']
#DF['Percent_Diff'] = DF['Diff R2']/DF['R2_max']
#DF['SNR 1'] = DF['SNR 1'].replace([np.inf, -np.inf], np.nan, inplace=True)#.dropna(subset=["SNR 1", "SNR "], how="all")
#DF['SNR 2'] = DF['SNR 2'].replace([np.inf, -np.inf], np.nan, inplace=True)#.dropna(subset=["SNR 1", "SNR "], how="all")

DF = DF.replace(-np.inf, np.nan)
DF = DF.dropna(axis=0)

In [14]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import use
use("Qt5Agg")
#for R1_min in range(1,int(max(DF['R1_min']))):

R = DF
#print(R)
fig = plt.figure()
ax = plt.axes(projection="3d")
ax._facecolors2d = ax._facecolor
  
surf1 = ax.plot_trisurf(R['SNR 1'],R['SNR 2'],R['Sumrate'],label = "Rate Near User + Rate Far User", color= 'orangered')
#surf2 =ax.plot_trisurf(R['Distancia 1'],R['Distancia 2'],R['R2_max'],label = "Max Rate (OMA)", color= 'navy')
surf3 =ax.plot_trisurf(R['SNR 1'],R['SNR 2'],R['R2'],label = "Rate Far User (NOMA)", color= 'lime')
surf4 =ax.plot_trisurf(R['SNR 1'],R['SNR 2'],R['R1_min'], label = "Rate Near User (NOMA)", color= 'darkcyan')
ax.set_xlim3d(-20, 250)
ax.set_ylim3d(-20, 250)
ax.set_zlim3d(0, 100)

ax.set_zlabel('bps/Hz')
ax.set_xlabel('Near User SNR (dB)')
ax.set_ylabel('Far User SNR (dB)')
#ax.set_zticks(np.arange(0,20,1))
ax.view_init(30, 135)
        
plt.title(f'44 dBm, R1_min = {R1_min} bps/Hz')
        
plt.show()

In [11]:
for 
if DF['Both QoS Satisfied'] == 'Yes':



SyntaxError: invalid syntax (<ipython-input-11-418b7f3845d2>, line 1)

In [ ]:
R2